# 네이버 영화 리뷰 페이지에 접속


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# requests 패키지 불러오기 (pip install requests 설치 필요)
import requests

# 네이버 영화 사이트 - 아바타: 물의 길 리뷰 페이지 
url = "https://movie.naver.com/movie/bi/mi/review.naver?code=74977"

# requests.get
resp = requests.get(url)
print(resp)

<Response [200]>


In [ ]:
# 서버 응답 중에서 소스코드 부분을 출력
resp.text[150:500]

'ng="ko">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta http-equiv="imagetoolbar" content="no">\r\n<title>아바타: 물의 길 : 네이버 영화</title>\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t\r\n\t\t\r\n\t\r\n\r\n\r\n\r\n<meta property="me2:image" content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg"/>\r\n<meta property="me2:post_tag" '

# BeautifulSoup으로 HTML 해석하기

In [ ]:
# BeautifulSoup 함수 불러오기 
from bs4 import BeautifulSoup 

# BeautifulSoup 함수로, HTML 문서 구조를 파싱 
soup = BeautifulSoup(resp.text, 'html.parser')

# 파싱한 내용을 담고 있는 객체의 자료형 확인
print(type(soup))

<class 'bs4.BeautifulSoup'>


## 1) 영화 제목

In [ ]:
# <title>아바타: 물의 길 : 네이버 영화</title>

# title 태그 이름을 활용하여 영화 제목이 포함되어 있는 요소를 찾습니다.
title_tag = soup.find(name='title')
print(title_tag)

# 텍스트 부분만 추출합니다.
title_text = title_tag.get_text()
print(title_text)

<title>아바타: 물의 길 : 네이버 영화</title>
아바타: 물의 길 : 네이버 영화


## 2) 리뷰 갯수


In [ ]:
# <span class="cnt">총<em>110</em>건</span>

# span 태그의 class 속성값을 활용하여 리뷰 갯수가 포함되어 있는 요소를 찾습니다.
count_tag = soup.find(name='span', attrs={'class':'cnt'})
print("span 태그: ", count_tag)

# count_tag 요소에서 em 태그 부분을 찾습니다. 
count_tag = count_tag.find(name='em')
print("em 태그: ", count_tag)

# 텍스트 부분만 추출합니다.
count_text = count_tag.get_text()
print("텍스트: ", count_text)

span 태그:  <span class="cnt">총<em>71</em>건</span>
em 태그:  <em>71</em>
텍스트:  71


## 3) 리뷰 목록


In [ ]:
# <ul class="rvw_list_area">

# ul 태그의 class 속성값을 활용하여 리뷰 제목과 링크가 포함되어 있는 요소를 찾습니다.
review_list_tag = soup.find(name='ul', attrs={'class':'rvw_list_area'})

# review_list_tag 요소에 포함된 li 태그를 모두 찾습니다. 
review_list_tags = review_list_tag.find_all(name='li')
print("li 태그의 수: ", len(review_list_tags))
print("\n")

# 첫 번째 원소를 출력합니다. 
print(review_list_tags[0])

li 태그의 수:  10


<li>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4061113)"><strong>&lt;아바타 2&gt; 주인공 "네이티리" 캐릭터 컨셉 아트 이미지 대공개... </strong></a>
<span class="user"><a href="#" onclick="clickcr(this, 'rli.uid', '', '', event); showReviewListByNid('4061113');">insu****</a> <em>2015.09.30</em><em>추천 189</em></span>
<p><a href="#" onclick="clickcr(this, 'rli.content', '', '', event); showReviewDetail(4061113);">        아바타 2      감독   제임스 카메론   출연   시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴   개봉   2017 미국      리뷰보기      ​*** 아바타 2 (2017) ***장르: Sci-Fi, 판타지, 모험, 액션감독: 제임스 카메론주연:...</a></p>
</li>


In [ ]:
# 첫 번째 아이템에서 리뷰 제목(rli.title), 사용자(rli.uid), 내용(rli.content) 데이터를 추출합니다.

review_title = review_list_tags[0].find_all('a')[0].get_text()
print("제목:", review_title, "\n")

review_uid = review_list_tags[0].find_all('a')[1].get_text()
print("사용자:", review_uid, "\n")

review_content = review_list_tags[0].find_all('a')[2].get_text()
print(r"내용:", review_content, "\n")

제목: <아바타 2> 주인공 "네이티리" 캐릭터 컨셉 아트 이미지 대공개...  

사용자: insu**** 

내용:         아바타 2      감독   제임스 카메론   출연   시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴   개봉   2017 미국      리뷰보기      ​*** 아바타 2 (2017) ***장르: Sci-Fi, 판타지, 모험, 액션감독: 제임스 카메론주연:... 



In [ ]:
# 각 리뷰 페이지로 연결할 nid 값(예: 4808692)을 포함하는 Javascript 코드를 추출합니다.

review_nid = review_list_tags[0].find('a').get('onclick')
review_nid

"clickcr(this, 'rli.title', '', '', event); showReviewDetail(4061113)"

In [ ]:
# Javascript 코드에서 7자리 숫자를 추출합니다. (정규식 re 모듈 활용)

import re

review_nid = re.findall('\d{7}', review_nid)[0]
review_nid

'4061113'

In [ ]:
# URL을 조합합니다.

review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=74977&order=#tab"
review_url

'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4061113&code=74977&order=#tab'

In [ ]:
# 리뷰를 반복문으로 조회하면서, 리뷰 제목(rli.title), 사용자(rli.uid), 상세 페이지 url 값을 추출합니다.
title_list = []
uid_list = []
url_list = []

from tqdm import tqdm

# page 1~8까지 1씩 증가하면서 조회
for page in tqdm(range(1,9)):

  URL = f"https://movie.naver.com/movie/bi/mi/review.naver?code=74977&page={page}"

  resp = requests.get(URL)
  soup = BeautifulSoup(resp.text, 'html.parser')

  review_list_tag = soup.find(name='ul', attrs={'class':'rvw_list_area'})
  review_list_tags = review_list_tag.find_all(name='li')

  print(len(review_list_tags))


  for li_tag in review_list_tags:
      
      review_title = li_tag.find_all('a')[0].get_text()
      title_list.append(review_title)
      
      review_uid = li_tag.find_all('a')[1].get_text()
      uid_list.append(review_uid)
      
      review_nid = re.findall('\d{7}', li_tag.find('a').get('onclick'))[0]
      review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=74977&order=#tab"
      url_list.append(review_url)    
    
# 추출된 아이템의 수량
print(len(title_list))
print(len(uid_list))
print(len(url_list))

 12%|█▎        | 1/8 [00:00<00:04,  1.52it/s]

10


 25%|██▌       | 2/8 [00:01<00:03,  1.51it/s]

10


 38%|███▊      | 3/8 [00:01<00:03,  1.53it/s]

...

10


 88%|████████▊ | 7/8 [00:04<00:00,  1.48it/s]

10


100%|██████████| 8/8 [00:05<00:00,  1.51it/s]

1
71
71
71


In [ ]:
# 리스트 출력
print(url_list)

['https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4061113&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=2415599&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4469161&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=3330958&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4312454&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=2860659&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4496289&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4417995&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=2204846&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=3638168&code=74977&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4529246&code=74977&order=#tab', 'https://

## 4) 리뷰 상세 페이지


In [ ]:
# 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
resp_text = requests.get(url_list[0], headers={'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
soup_text = BeautifulSoup(resp_text.text, 'html.parser')

# 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

# 텍스트 부분만 추출합니다.
review_text = review_text_tag.get_text()
print(review_text)






아바타 2

감독
제임스 카메론
출연
시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴
개봉
2017 미국

리뷰보기


​*** 아바타 2 (2017) ***장르: Sci-Fi, 판타지, 모험, 액션감독: 제임스 카메론주연: 조 셀다나 (네이티리 역), 샘 워싱턴 (제이크 설리 역), 시고니 위버 (그레이스 어거스틴 역), 스티븐 랭 (마일즈 쿼리치 역)...국적: 미국북미 개봉일: 2017년 12월 25일2017년 12월 25일 북미 개봉작으로 기대를 모으고 있는 영화 아바타 2...특히나 주인공 "네이티리" 의 캐릭터 이미지가 공개가 되어서 올려봅니다.배우 조 샐다나와 샘 워싱턴, 시고니 위버 그리고 스티븐 랭까지 전편에서 출연했었던 배우들이 그대로 출연을 하고 있어서더욱 기대가 되는 영화인 것 같습니다. 아무튼 제임스 카메론 감독의 영화 아바타 2편을 시작으로2018년 아바타 3와 2019년 아바타 4까지 진행되고 있어서 앞으로 더욱 기대되는 영화 아바타인 것 같습니다.



## 5) 리뷰를 수집하여 List로 정리

In [ ]:
import time

# url 반복하여 텍스트를 추출하고 리스트에 추가

text_list = []

for url in url_list:

    # 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
    resp_text = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
    soup_text = BeautifulSoup(resp_text.text, 'html.parser')

    # 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
    review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

    # 텍스트 부분만 추출합니다.
    review_text = review_text_tag.get_text()
    text_list.append(review_text)
    
    # 대기 시간을 추가합니다. (서버에 과도한 호출이 되지 않도록 유의) 
    time.sleep(2)
    
    
# 추출된 아이템의 수량
print(len(text_list))

71


## 6) 판다스 데이터프레임으로 정리

In [ ]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가 
dict_data = {
    'title' : title_list,
    'user' : uid_list,
    'review' : text_list   
}

# 판다스 데이터프레임으로 변환
import pandas as pd

df_data = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_data

,title,user,review
0,"<아바타 2> 주인공 ""네이티리"" 캐릭터 컨셉 아트 이미지 대공개...",insu****,"\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티..."
1,2014년에 이글을 읽는사람들에게..,xnzm****,"\n2014년에 아바타2가 개봉하다니ㅋ\n \n아바타1,, 정말 재밌게 봤었는데\n..."
2,"[필독, 스압] 위대한 지휘관, 쿼리치대령을 추모하며...",alwa****,\n영화 아바타는.끝까지 우리 인류의 번영과 이익을 위해 사투를 벌이다 죽어간 위대...
3,"아바타2 3,4 제작소식 모음",soqu****,\n \n \n \n<< 아바타 속편 제작소식 모음 >>\n \n\n \n \n...
4,아바타2 스토리,doub****,\n전작의 경우 cg는 혁명적이었지만제임스 카메론이 14년동안 구상한거 치곤 스토리...
...,...,...,...
66,"제임스 카메론 감독, <아바타 2>의 촬영이 마무리되었음을 밝히다.",c106****,\n 많은 분들의 기대(와 우려)를 한 몸에 받고 있는 <아바타 2>의 촬영...
67,"제임스 카메론 감독, <아바타 2>에서 판도라 행성의 사실적인 수중 세계를 보게 될...",c106****,\n <아바타> 시퀄 제작에 열중하고 있는 제임스 카메론 감독이 <아바타 2...
68,아바타 2 - 2020년 12월 개봉 소식 = 2020년 개봉예정영화,inee****,"\n아바타 2 (Avatar 2, 2020)\n\n아바타 2\n\n감독\n제임스 카..."
69,"<아바타> 시퀄, 올봄 뉴질랜드에서 라이브 액션 촬영 시작!!!",c106****,"\n 역대 월드와이드 최고 흥행 기록(※27억8,796만불)을 가지고 있는 ..."


In [ ]:
# CSV 파일로 저장
df_data.to_csv("naver_review_Avatar.csv")